<h1 align="center"> 海洋数据分析 第十二讲</h1>
<h3 align="right">主讲人: 陈笔澄</h3>
<h2>目录<span class="tocSkip"></span></h2>

- ## [制作动画准备--FFmpeg](#sec:ffmpeg)
- ## [动画制作流程--matplotlib.animation](#sec:step)

## 动画学习目的:
- ### <font  color="gray">数据获取 数据分析</font> <font size=20>成果展示</font>
- ### 使用更为形象的方式展示成果
- ### 加强对自然现象的理解

## 帮助链接: 
- ### Animation官方网站: https://matplotlib.org/stable/api/animation_api.html#
- ### FFmpeg官方网站: https://ffmpeg.org

---
---

## 

---
---

## 🗂 <a id = "sec:ffmpeg"> 制作动画准备--FFmpeg

---

## ❓ FFmpeg

## 💡 FFmpeg是一种多媒体处理套件

- ### 具备解译、编码、转义、过滤各种视频格式

- ### 多平台支持，如windows, Mac OS, Linux

---

## ？ 如何安装

## 💡 Anaconda UI界面下直接安装

## 💡 Anaconda命令行下输入`conda install -c conda-forge ffmpeg`

---
---

## 

---
---

## 🗂 <a id = "sec:step"> 动画制作流程--matplotlib.animation

---

## ❓ 动画片是怎么制作的

## 💡 是用一帧一帧的图像连续播放得来的

## 💡 我们制作动画也一样
- ### 前置能力: 使用Matplotlib前置能力 ✔️
- ### 本讲内容: 将所画的图逐帧拼接 🤔

---

## ✏️ 首先我们回到如何画一张海洋上层朗缪尔湍流上

### 前置工作

In [ ]:
## 模块
import matplotlib as mpl 
import matplotlib.pyplot as plt 
import matplotlib.colors as colors
from matplotlib import cm
from matplotlib.ticker import MultipleLocator
from matplotlib import animation
import numpy as np
from IPython.display import HTML

## 自定义变量
# 文件
fn = "./attachment/w_cross.npz"

# 图片
margin = dict(left=0.1, right=0.95, bottom=0.1, top=0.98,
              hspace=0.1, wspace=0.1)
levels_w = np.linspace(-2, 2, 21)
cmap_w = cm.get_cmap("RdYlBu_r")
xspacing = 100
yspacing = 100
zspacing = 25

## 

### 数据获取
- ### 受同学们启发，使用了将字符串作为变量名的特殊技术

In [ ]:
## 读取数据
data = np.load(fn)
keys = list(data.keys())
print("Keys in the dataset:", keys)

for key in keys:
    vars()[key] = data[key]

In [ ]:
## 如vars()函数使用字符串‘dx’创建了了变量dx
print("dx = ", dx)

## 

### 数据处理

In [ ]:
## 创建坐标
x1d = np.arange(dx/2, lx, dx)
y1d = np.arange(dy/2, ly, dy)
z1d = np.arange(0, -lz, -dz)

## 

### 成果展示

In [ ]:
# 每一帧的画布信息
ifig = 0
fig = plt.figure(ifig, figsize=[8, 8])
gs = fig.add_gridspec(2, 2, width_ratios=(2, 1), height_ratios=(2, 1),
    **margin)
xmaxLocator = MultipleLocator(xspacing)
ymaxLocator = MultipleLocator(yspacing)
zmaxLocator = MultipleLocator(zspacing)
xlims = (0, lx)
ylims = (0, ly)
zlims = (-lz, 0)

# 将第一个时刻的垂直运动w作为示意图
it = 0
        
# w 水平截面图
ax = fig.add_subplot(gs[0, 0])
ax.set_aspect('equal')

cax = ax.contourf(x1d, y1d, w_xy[it, :, :],
    levels=levels_w, cmap=cmap_w, extend='both')

# 格式调整
ax.set_ylabel('y (m)')
ax.set_xticklabels([])
ax.xaxis.set_major_locator(xmaxLocator)
ax.yaxis.set_major_locator(ymaxLocator)
ax.set_xlim(xlims)
ax.set_ylim(ylims)



## w xz截面图
ax = fig.add_subplot(gs[1, 0])
ax.set_aspect(1.666)

ax.contourf(x1d, z1d, w_xz[it, :-1, :],
    levels=levels_w, cmap=cmap_w, extend='both')

# 格式调整
ax.set_xlabel('x (m)')
ax.set_ylabel('z (m)')
ax.xaxis.set_major_locator(xmaxLocator)
ax.yaxis.set_major_locator(zmaxLocator)
ax.set_xlim(xlims)
ax.set_ylim(zlims)



## w数据yz截面
ax = fig.add_subplot(gs[0, 1])
ax.set_aspect(0.6)
ax.contourf(z1d, y1d, w_yz[it, :-1, :].T,
    levels=levels_w, cmap=cmap_w, extend='both')

# 格式调整
ax.set_xlabel('z (m)')
ax.set_yticklabels([])
ax.xaxis.set_major_locator(zmaxLocator)
ax.yaxis.set_major_locator(ymaxLocator)
ax.set_xlim(zlims)
ax.set_ylim(ylims)
ax.invert_xaxis()

# 添加色阶图
cbar_ax = plt.axes([0.82, 0.1, 0.05, 0.2])
cbar = fig.colorbar(cax, cax=cbar_ax, ticks=levels_w[::5])
cbar.ax.set_title("$w/u_*$", ha='center', va='bottom')

# plt.show()

---

## ✏️ 下一步: 重新成果展示，将实现循环作画并保存为动画的每一帧

### 1. 需要创建视频写入器
- ### 使用`matplotlib.animation`底下的这个多媒体套件(`XXXWriter`)
- ### 这里展示的是FFMpeg(FFMpegWriter)套件的使用，常用套件还包含Pillow, HTML等

In [ ]:
# 视频基本信息
FFMpegWriter = animation.FFMpegWriter # 制作视频的工具FFMpeg
metadata = dict() # 可以定义一些视频元数据，比如作者，视频名称
writer = FFMpegWriter(fps=10, metadata=metadata,
    extra_args=['-vcodec', 'h264', '-pix_fmt', 'yuv420p'])

- ### fps: frame per-second, 帧率(每一秒播放多少帧)
- ### metadata: 多媒体元数据(如作者，视频名称等)，可根据需求随意定义
- ### vcodec: 编码格式，常用h264
- ### pix_fmt: 像素格式，常用yuv420p

## 

### 2. 保存每一帧图像
- ### 使用`writer.saving(figure, video_name, dpi)`打开视频写入器
- ### 使用`writer.grab_frame()`抓取每一帧图像

In [ ]:
# 每一帧的画布信息
ifig = 0
fig = plt.figure(ifig, figsize=[8, 8])
gs = fig.add_gridspec(2, 2, width_ratios=(2, 1), height_ratios=(2, 1),
    **margin)
xmaxLocator = MultipleLocator(xspacing)
ymaxLocator = MultipleLocator(yspacing)
zmaxLocator = MultipleLocator(zspacing)
xlims = (0, lx)
ylims = (0, ly)
zlims = (-lz, 0)

# 逐帧作图
fn_ani = './image/langmuirTurbulence_2D.mp4'
### 第一个重要步骤: 打开视频写入器
with writer.saving(fig, fn_ani, 100):
### 第一个重要步骤: 打开视频写入器
    for it in range(nt):
        time = it*dt
        print("Creating the frame {:2d}... ".format(it+1))
        
        # w 水平截面图
        ax = fig.add_subplot(gs[0, 0])
        ax.set_aspect('equal')

        cax = ax.contourf(x1d, y1d, w_xy[it, :, :],
            levels=levels_w, cmap=cmap_w, extend='both')

        # 格式调整
        ax.set_ylabel('y (m)')
        ax.set_xticklabels([])
        ax.xaxis.set_major_locator(xmaxLocator)
        ax.yaxis.set_major_locator(ymaxLocator) 
        ax.set_xlim(xlims) 
        ax.set_ylim(ylims)



        ## w xz截面图
        ax = fig.add_subplot(gs[1, 0])
        ax.set_aspect(1.6666)
        
        ax.contourf(x1d, z1d, w_xz[it, :-1, :],
            levels=levels_w, cmap=cmap_w, extend='both')

        # 格式调整
        ax.set_xlabel('x (m)')
        ax.set_ylabel('z (m)')
        ax.xaxis.set_major_locator(xmaxLocator)
        ax.yaxis.set_major_locator(zmaxLocator)
        ax.set_xlim(xlims)
        ax.set_ylim(zlims)



        ## w数据yz截面
        ax = fig.add_subplot(gs[0, 1])
        ax.set_aspect(0.6)
        ax.contourf(z1d, y1d, w_yz[it, :-1, :].T,
            levels=levels_w, cmap=cmap_w, extend='both')

        # 格式调整
        ax.set_xlabel('z (m)')
        ax.set_yticklabels([])
        ax.xaxis.set_major_locator(zmaxLocator)
        ax.yaxis.set_major_locator(ymaxLocator)
        ax.set_xlim(zlims)
        ax.set_ylim(ylims)
        ax.invert_xaxis() # 反转x坐标

        # 添加色阶图
        cbar_ax = plt.axes([0.82, 0.1, 0.05, 0.2])
        cbar = fig.colorbar(cax, cax=cbar_ax, ticks=levels_w[::5])
        cbar.ax.set_title("$w/u_*$", ha='center', va='bottom')
        
        ### 第二个重要步骤: 作图成功后，抓取图像保存为一帧
        writer.grab_frame()
        ### 第二个重要步骤: 作图成功后，抓取图像保存为一帧

--- 

## ✏️ 二维动画不够符合真实世界的视角 &rarr; 三维动画

- ### 指定子图投影为三维: `ax = fig.add_subplot(*, projection='3d')`
- ### 等值线图是指定平面方向，如`cax = ax.contourf(*, zdir='z')`
- ### 使用`contourf()`特别注意坐标标量与所画变量w的位置关系(见示例)
- ### ax.view_init(elev, azim, roll)调整相机(视角)位置

<img src="https://matplotlib.org/stable/_images/mplot3d_view_angles.png" width=600>

In [ ]:
## 创建一个三维湍流动画
# 每一帧的画布信息
ifig += 1
fig = plt.figure(ifig, figsize=[8, 8])
gs = fig.add_gridspec(1, 1, **margin)
xmaxLocator = MultipleLocator(xspacing)
ymaxLocator = MultipleLocator(yspacing)
zmaxLocator = MultipleLocator(zspacing)
xlims = (0, lx)
ylims = (0, ly)
zlims = (-lz, 0)

# 逐帧作图
z3d, y3d, x3d=np.meshgrid(z1d, y1d, x1d, indexing='ij')

fn_ani = './image/langmuirTurbulence_3D.mp4'
with writer.saving(fig, fn_ani, 100):
    for it in range(nt):
        time = it*dt
        print("Creating the frame {:2d}... ".format(it+1))
        
        # w 水平截面图
        ### 投影必须为3维
        ax = fig.add_subplot(gs[0, 0], projection='3d')
         ### 投影必须为3维
        ax.set_aspect('equal')
        
        ### 特别注意坐标标量与所画变量w的位置关系 ###
        cax = ax.contourf(x3d[iz, :, :], y3d[iz, :, :], w_xy[it, :, :],
           levels=levels_w, cmap=cmap_w, extend='both', zdir='z', offset=-iz)
        
        ax.contourf(x3d[:, iy, :], w_xz[it, :-1, :], z3d[:, iy, :],
            levels=levels_w, cmap=cmap_w, extend='both', zdir='y', offset=0)

        ax.contourf(w_yz[it, :-1, :], y3d[:, :, ix], z3d[:, :, ix],
           levels=levels_w, cmap=cmap_w, extend='both', zdir='x', offset=lx)
        ### 特别注意坐标标量与所画变量w的位置关系 ###
        
        # 格式调整
        ax.set(xlim=xlims, ylim=ylims, zlim=zlims)
        ### 控制相机角度
        ax.view_init(45, -60, 0)
        ### 控制相机角度
        ax.set_xlabel('x (m)')
        ax.set_ylabel('y (m)')
        ax.set_zlabel('z (m)')

        # 添加色阶图
        cbar_ax = plt.axes([0.88, 0.1, 0.05, 0.2])
        cbar = fig.colorbar(cax, cax=cbar_ax, ticks=levels_w[::5])
        cbar.ax.set_title("$w/u_*$", ha='center', va='bottom')
        
        # 抓取图片
        writer.grab_frame()

---
---

## 

---
---

## 最后，我们又多了一节课。。。老师得想想讲什么topic，希望同学们踊跃提建议。

---
---